In [ ]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0  25370      0 --:--:-- --:--:-- --:--:-- 25370
OK
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 http

In [ ]:
!apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tensorflow-model-server
0 upgraded, 1 newly installed, 0 to remove and 68 not upgraded.
Need to get 210 MB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server amd64 tensorflow-model-server all 2.3.0 [210 MB]
Fetched 210 MB in 3s (79.4 MB/s)
Selecting previously unselected package tensorflow-model-server.
(Reading database ... 144793 files and directories currently installed.)
Preparing to unpack .../tensorflow-model-server_2.3.0_all.deb ...
Unpacking tensorflow-model-server (2.3.0) ...
Setting up tensorflow-model-server (2.3.0) ...


### Start running TensorFlow Serving
This is where we start running TensorFlow Serving and load our model. After it loads we can start making inference requests using REST. There are some important parameters:

* rest_api_port: The port that you'll use for REST requests.
* model_name: You'll use this in the URL of REST requests. It can be anything.
* model_base_path: This is the path to the directory where you've saved your model.

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
%cd MyDrive/TFX/

/gdrive/MyDrive/TFX


In [ ]:
!ls

exported_model_latest


In [ ]:
%cd MyDrive/BDS7245/case_study_4/

/gdrive/MyDrive/BDS7245/case_study_4


In [ ]:
ls exported_model_latest/9/

assets/  saved_model.pb  variables/


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# TensorFlow and tf.keras
print("Installing dependencies for Colab environment")
!pip install -Uq grpcio==1.26.0

import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

print('TensorFlow version: {}'.format(tf.__version__))

Installing dependencies for Colab environment
     |████████████████████████████████| 2.4MB 5.9MB/s 
TensorFlow version: 2.3.0


In [ ]:
os.environ["MODEL_DIR"] = MODEL_DIR = "/gdrive/My Drive/TFX/exported_model_latest"

In [ ]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=exported_model_latest \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

Starting job # 2 in a separate thread.


In [ ]:
!tail server.log

2020-12-04 06:03:10.737817: I tensorflow_serving/model_servers/server_core.cc:575]  (Re-)adding model: exported_model_latest
2020-12-04 06:03:10.847907: I tensorflow_serving/core/basic_manager.cc:739] Successfully reserved resources to load servable {name: exported_model_latest version: 9}
2020-12-04 06:03:10.848052: I tensorflow_serving/core/loader_harness.cc:66] Approving load for servable version {name: exported_model_latest version: 9}
2020-12-04 06:03:10.848126: I tensorflow_serving/core/loader_harness.cc:74] Loading servable version {name: exported_model_latest version: 9}
2020-12-04 06:03:10.848979: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:31] Reading SavedModel from: /gdrive/My Drive/TFX/exported_model_latest/9
2020-12-04 06:03:11.526226: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:54] Reading meta graph with tags { serve }
2020-12-04 06:03:11.526535: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:234] Reading SavedModel deb

In [ ]:
import tempfile

tempfile.gettempdir()

'/tmp'

In [ ]:
%%bash  
ls

exported_model_latest
server.log


In [ ]:
%%bash
realpath exported_model_latest

/gdrive/.shortcut-targets-by-id/100l7BvgP9GvwqwAZtVIj5K-s-Zt7sdkK/exported_model_latest


In [ ]:
import json
data = json.dumps({"signature_name": "serving_default", "instances": ["it was good"]})
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))

Data: {"signature_name": "serving_default", "instances": ... e": "serving_default", "instances": ["it was good"]}


In [ ]:
!pip install -q requests

import requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/exported_model_latest:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)

In [ ]:
predictions

{'error': "Could not parse example input, value: 'it was good'\n\t [[{{node ParseExample/ParseExampleV2}}]]"}

In [ ]:
import requests
import tensorflow as tf
import base64

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def serialize_text(text):
    example = tf.train.Example(features=tf.train.Features(feature={
        'text': _bytes_feature(text)
        }))
    serialized_example = example.SerializeToString()
    return serialized_example

model_server_url = 'http://localhost:8501/v1/models/exported_model_latest:predict'

def predict(text):
    example = serialize_text(text)
    json_data = {
      "signature_name":"serving_default",
      "instances":[
        {
          "examples":{"b64": base64.b64encode(example).decode('utf-8')}
        }
      ]
    }
    resp = requests.post(model_server_url, json=json_data)
    return resp.json()

In [ ]:
text = b"This is the worst show ever produced for TV. Each episode is a flop. The casting, the writing, the timing are all really bad."
print(predict(text))

{'predictions': [[0.033854723]]}
